In [35]:
import os
import shutil
import tempfile
from pathlib import Path

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType


# 🔧 CONFIGURATION -------------------------------------------------------------

# Source: file-based MLflow store (your current ./mlruns directory)
SOURCE_URI = "./mlruns"  # or "file:./mlruns"

# Destination: SQLite backend store
# This assumes you want a local SQLite DB. Adjust the path as you like.
DEST_URI = "sqlite:///mlruns.db"

# Where to store artifacts for the DESTINATION runs
DEST_ARTIFACT_ROOT = "./mlruns_sqlite_artifacts"


# 🚀 MIGRATION LOGIC -----------------------------------------------------------

def ensure_dest_artifact_root():
    Path(DEST_ARTIFACT_ROOT).mkdir(parents=True, exist_ok=True)
    return Path(DEST_ARTIFACT_ROOT).absolute().as_uri()


def migrate():
    # --- Connect clients ---
    print(f"🔹 Connecting to Source: {SOURCE_URI}")
    src_client = MlflowClient(tracking_uri=SOURCE_URI)

    print(f"🔹 Connecting to Destination (backend DB): {DEST_URI}")
    dest_client = MlflowClient(tracking_uri=DEST_URI)

    dest_artifact_root_uri = ensure_dest_artifact_root()
    print(f"🔹 Destination artifact root: {dest_artifact_root_uri}")

    # --- Migrate experiments ---
    print("🔹 Listing source experiments...")
    src_experiments = src_client.search_experiments(
        view_type=ViewType.ACTIVE_ONLY
    )

    exp_id_map = {}  # src_exp_id -> dest_exp_id

    for src_exp in src_experiments:
        print(f"\n📁 Experiment: '{src_exp.name}' (id={src_exp.experiment_id})")

        # Check if experiment already exists in destination
        dest_exp = dest_client.get_experiment_by_name(src_exp.name)
        if dest_exp is None:
            dest_exp_id = dest_client.create_experiment(
                name=src_exp.name,
                artifact_location=os.path.join(dest_artifact_root_uri, src_exp.name)
            )
            print(f"  ✅ Created destination experiment id={dest_exp_id}")
        else:
            dest_exp_id = dest_exp.experiment_id
            print(f"  ↪ Using existing destination experiment id={dest_exp_id}")

        exp_id_map[src_exp.experiment_id] = dest_exp_id

        # --- Migrate runs for this experiment ---
        migrate_experiment_runs(
            src_client=src_client,
            dest_client=dest_client,
            src_experiment_id=src_exp.experiment_id,
            dest_experiment_id=dest_exp_id,
        )


def migrate_experiment_runs(src_client, dest_client, src_experiment_id, dest_experiment_id):
    print(f"  🔹 Migrating runs for experiment {src_experiment_id} → {dest_experiment_id}")

    page_token = None
    total_runs = 0

    while True:
        runs_page = src_client.search_runs(
            experiment_ids=[src_experiment_id],
            filter_string="",
            run_view_type=ViewType.ACTIVE_ONLY,
            max_results=1000,
            page_token=page_token,
        )

        if len(runs_page) == 0:
            break

        for src_run in runs_page:
            total_runs += 1
            print(f"    ▶ Migrating run {src_run.info.run_id}")
            migrate_single_run(
                src_client=src_client,
                dest_client=dest_client,
                src_run=src_run,
                dest_experiment_id=dest_experiment_id,
            )

        page_token = runs_page.token
        if page_token is None:
            break

    print(f"  ✅ Migrated {total_runs} runs from experiment {src_experiment_id}")


def migrate_single_run(src_client, dest_client, src_run, dest_experiment_id):
    src_run_id = src_run.info.run_id

    # --- Create destination run with same metadata ---
    tags = dict(src_run.data.tags) if src_run.data.tags is not None else {}
    # Optional: keep a record of the original run id
    tags["migrated_from_run_id"] = src_run_id

    run_name = tags.get("mlflow.runName", None)

    dest_run = dest_client.create_run(
        experiment_id=dest_experiment_id,
        start_time=src_run.info.start_time,
        tags=tags,
        run_name=run_name,
    )
    dest_run_id = dest_run.info.run_id

    # --- Copy params ---
    for k, v in src_run.data.params.items():
        dest_client.log_param(dest_run_id, k, v)

    # --- Copy full metric history ---
    for metric_key in src_run.data.metrics.keys():
        history = src_client.get_metric_history(src_run_id, metric_key)
        for m in history:
            dest_client.log_metric(
                run_id=dest_run_id,
                key=metric_key,
                value=m.value,
                step=m.step,
                timestamp=m.timestamp,
            )

    # --- Copy artifacts ---
    copy_artifacts_between_runs(src_client, dest_client, src_run_id, dest_run_id)

    # --- Preserve final status and end time ---
    dest_client.set_terminated(
        run_id=dest_run_id,
        status=src_run.info.status,
        end_time=src_run.info.end_time,
    )


def copy_artifacts_between_runs(src_client, dest_client, src_run_id, dest_run_id):
    """
    Download all artifacts from src_run_id and re-log them to dest_run_id.
    """

    # Download to a temporary directory
    tmpdir = tempfile.mkdtemp(prefix="mlflow_migrate_")
    try:
        # "" means "root of artifact store" – download everything
        local_artifact_path = src_client.download_artifacts(
            run_id=src_run_id,
            path="",
            dst_path=tmpdir,
        )

        # Re-log everything to the destination run
        dest_client.log_artifacts(
            run_id=dest_run_id,
            local_dir=local_artifact_path,
        )
    finally:
        shutil.rmtree(tmpdir, ignore_errors=True)


# -----------------------------------------------------------------------------


if __name__ == "__main__":
    migrate()
    print("\n🎉 Migration complete.")


🔹 Connecting to Source: ./mlruns
🔹 Connecting to Destination (backend DB): sqlite:///mlruns.db
🔹 Destination artifact root: file:///Users/lucascruz/Documents/GitHub/rl-research/mlruns_sqlite_artifacts
🔹 Listing source experiments...

📁 Experiment: 'test' (id=769745324565246657)
  ↪ Using existing destination experiment id=11
  🔹 Migrating runs for experiment 769745324565246657 → 11
    ▶ Migrating run e2f6fc9487df460482082c4d32a621d4


    ▶ Migrating run b3e3338204094a54bfd9a30b756460bd


  ✅ Migrated 2 runs from experiment 769745324565246657

🎉 Migration complete.
